<a href="https://colab.research.google.com/github/juuliana/imersao-gemini-api-alura/blob/main/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o SDK do Google

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

#Substitua a SECRET_KEY pela sua chave do Google AI Studio
SECRET_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=SECRET_KEY)

Listar os modelos disponíveis

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Trabalhando com embedding

In [ ]:
text = "Olá"
result = genai.embed_content(model="models/embedding-001", content=text)
embedding = result['embedding']
print(embedding)

In [ ]:
print(len(embedding))

768


Trabalhando com uma lista de embedding

In [ ]:
result = genai.embed_content(model="models/embedding-001",
                             content=[
                                "Hello",
                                "Hello World",
                                "World"
                             ])

for embedding in result['embedding']:
  print(str(embedding)[:50], '... TRIMMED')

Aplicando o embedding

In [ ]:
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
               "\n",
               "Artigo completo:\n",
               "\n",
               "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001", content=sample_text, title=title, task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

Listagem de documentos que serão buscados

In [ ]:
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central.  Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuir o fluxo de ar. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite que você selecione o modo desejado. Os modos disponíveis são: Auto: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Cool (Frio): O carro soprará ar frio para dentro do carro. Heat: O carro soprará ar quente para dentro do carro. Defrost (Descongelamento): O carro soprará ar quente no para-brisa para descongelá-lo."}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle climático. Para usar a tela sensível ao toque, basta tocar no ícone desejado.  Por exemplo, você pode tocar no ícone \"Navigation\" (Navegação) para obter direções para o seu destino ou tocar no ícone \"Music\" (Música) para reproduzir suas músicas favoritas."}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


Criando o modelo

In [ ]:
model = "models/embedding-001"

In [ ]:
def embed_fn(title, text):
  return genai.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")['embedding']

In [ ]:
df['Embeddings'] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.011010795, -0.026731547, -0.036728486, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.015715627, -0.040427547, 0.011117627, 0.002..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.009390755, -0.022475218, -0.0024638234, 0...."


In [ ]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")['embedding']

  produtos_escalares = np.dot(np.stack(df['Embeddings']), embedding_da_consulta)
  index = np.argmax(produtos_escalares)

  return df.iloc[index]['Conteudo']

In [ ]:
consulta = "Como faço para trocar marchas em um carro do Google?"

response = gerar_e_buscar_consulta(consulta, df, model)
print(response)

Ajustando a resposta de uma forma descontraída

In [ ]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {consulta}"

generation_config = {
    'candidate_count': 1,
    'temperature': 0.5
}

model_2 = genai.GenerativeModel(model_name="gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)